In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install sktime neuralforecast pmdarima darts numpy==1.26.4

In [3]:
import os
os.chdir('engagement-prediction/src')

In [4]:
storage_path='/content/drive/MyDrive/thesis/'

In [5]:
import pandas as pd
import numpy as np
import os
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator
h=6
window=24
sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
file_path=os.path.join(storage_path,'data','influencer_sample010525.csv')
df= pd.read_csv(file_path)
#df= pd.read_csv('~/rug/thesis/data/influencer_sample010525.csv')
df=df.query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['publishedDate'])
df
df_temp= df.query('channel_uid=="51b52b775df43f1ea0abf5db8789446d"')[['published_date', 'video_plays']].dropna()
df_temp = df_temp.set_index('published_date').sort_index()
y= df_temp['video_plays']
y = y.sort_index()
y_pos = y.reset_index(drop=True)

#y = y.asfreq('D')
y

,video_plays
published_date,
2021-09-09 19:23:35+00:00,4187.0
2021-09-16 12:17:06+00:00,3216.0
2021-09-21 12:15:06+00:00,1914.0
2021-09-22 11:24:18+00:00,656.0
2021-09-25 19:34:55+00:00,1619.0
...,...
2024-12-27 14:01:46+00:00,2017.0
2025-01-07 13:34:13+00:00,2425.0
2025-01-10 10:32:02+00:00,1828.0


In [6]:
df['source'].value_counts()

,count
source,
instagram-content,136584
twitter-tweets,50484
tiktok-content,35057
facebook-content,10885
youtube-content,2949
tiktok-content-oauth,1602
instagram-content-oauth,734
tiktok-content-public,62


In [7]:
df.groupby('channel_uid').count()['publishedDate'].sort_values(ascending=False).head(10)

,publishedDate
channel_uid,
26bf145cbe9c3a10a53c619f05aa750f,32674
1058025cd111366f9d4d4af89ce1d8d0,24051
954328679b923a4fa7af327a2cb5b84d,13640
da9acec7669f3db79050175948471cde,9714
e04c6702e00a321a8f98624ffe7fd23e,6232
6b9e43bfbb843f07b99ec55767b87e67,5466
24a13f11774b37b2bb94e9ab25327bf7,4884
e6981573da773e0eb0493a0cf142e27c,4274
428271c836453fc2a6fd57748371c14d,3608


In [8]:
ids_test=df.query("""source in ['instagram-content','youtube-content','tiktok-content','facebook-content']""")\
    ['channel_uid'].drop_duplicates().sample(20, random_state=1).values
df_temp=df.query('channel_uid in @ids_test')
df_temp['source'].unique()

array(['tiktok-content', 'tiktok-content-oauth', 'instagram-content',
       'facebook-content', 'youtube-content', 'tiktok-content-public'],
      dtype=object)

In [9]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [10]:

multi_eval.run(h=6, initial_window=24, step_length=6,models=['MA(4)','AR(4)'],metric_cols=['potentialReach','engagements','video_views','comments','shares'])

summary_df = multi_eval.get_summary()

Evaluating models:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWa

In [11]:
multi_eval.get_summary().query('channel_uid=="e4786c9ac7d23a40aa6840e8d817642b"')

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,AR(4),0.001138,1.936054e-08,0.001310,e4786c9ac7d23a40aa6840e8d817642b,potentialReach,tiktok-content
1,MA(4),0.000005,8.205060e-11,0.000005,e4786c9ac7d23a40aa6840e8d817642b,potentialReach,tiktok-content
2,AR(4),143.192183,8.393631e-01,170.698311,e4786c9ac7d23a40aa6840e8d817642b,engagements,tiktok-content
3,MA(4),149.201755,8.639093e-01,177.957747,e4786c9ac7d23a40aa6840e8d817642b,engagements,tiktok-content
4,AR(4),0.000005,2.172788e+10,0.000005,e4786c9ac7d23a40aa6840e8d817642b,video_views,tiktok-content
5,MA(4),0.000005,2.172788e+10,0.000005,e4786c9ac7d23a40aa6840e8d817642b,video_views,tiktok-content
6,AR(4),15.067546,2.075668e+00,22.226020,e4786c9ac7d23a40aa6840e8d817642b,comments,tiktok-content
7,MA(4),14.800526,1.621518e+00,22.601690,e4786c9ac7d23a40aa6840e8d817642b,comments,tiktok-content
8,AR(4),13.673063,8.215688e+15,18.271457,e4786c9ac7d23a40aa6840e8d817642b,shares,tiktok-content
9,MA(4),15.633508,9.178537e+15,21.343366,e4786c9ac7d23a40aa6840e8d817642b,shares,tiktok-content


In [12]:
import datetime as dt
output_path=os.path.join(storage_path,'output',f"summary_df_{dt.datetime.now():%y%m%d%H%M}.csv")
summary_df.to_csv(output_path, index=False)
summary_df

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/thesis/cache'